## 第二章 pandas基础
### Ex1：口袋妖怪数据集
1.对`HP, Attack, Defense, Sp. Atk, Sp. Def, Speed`进行加总，验证是否为`Total`值。

In [1]:
import numpy as np
import pandas as pd
df = pd.read_csv('./data/pokemon.csv') #.set_index(['#'])
df.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65


In [37]:
sum(df[df.columns[5:]].sum(1)!=df.Total)

0

2. 对于`#`重复的妖怪只保留第一条记录，解决以下问题：
* 求第一属性的种类数量和前三多数量对应的种类
* 求第一属性和第二属性的组合种类
* 求尚未出现过的属性组合

In [48]:
df=df.drop_duplicates(['#'], keep='first')

print(df['Type 1'].nunique())
print(df['Type 1'].value_counts().index[:3])

df.drop_duplicates(['Type 1','Type 2'], keep='first')

L_full = [i+' '+j for i in df['Type 1'].unique() for j in (df['Type 1'].unique().tolist() + [''])]
L_part = [i+' '+j for i, j in zip(df['Type 1'], df['Type 2'].replace(np.nan, ''))]
res = set(L_full).difference(set(L_part))
len(res) 

18
Index(['Water', 'Normal', 'Grass'], dtype='object')


199

3. 按照下述要求，构造`Series`：

* 取出物攻，超过120的替换为`high`，不足50的替换为`low`，否则设为`mid`
* 取出第一属性，分别用`replace`和`apply`替换所有字母为大写
* 求每个妖怪六项能力的离差，即所有能力中偏离中位数最大的值，添加到`df`并从大到小排序

In [49]:
df['Attack'].mask(df['Attack']<=50,'low').mask(df['Attack']>120,'high').mask((50<=df['Attack'])&(df['Attack']<=120),'mid')

0       low
1       mid
2       mid
4       mid
5       mid
       ... 
793    high
794     mid
795     mid
797     mid
799     mid
Name: Attack, Length: 721, dtype: object

In [50]:
df['Type 1'].replace({i:str.upper(i) for i in df['Type 1'].unique()})
df['Type 1'].apply(lambda x:str.upper(x)).head()

0    GRASS
1    GRASS
2    GRASS
4     FIRE
5     FIRE
Name: Type 1, dtype: object

In [51]:
df['Deviation']=df[df.columns[5:11]].apply(lambda x:np.max((x-x.median()).abs()), 1)
df.sort_values('Deviation', ascending=False).head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Deviation
230,213,Shuckle,Bug,Rock,505,20,10,230,10,230,5,215.0
121,113,Chansey,Normal,NaN,450,250,5,5,35,105,50,207.5
261,242,Blissey,Normal,NaN,540,255,10,10,75,135,55,190.0
217,202,Wobbuffet,Psychic,NaN,405,190,33,58,33,58,33,144.5
223,208,Steelix,Steel,Ground,510,75,85,200,55,65,30,130.0


### Ex2：指数加权窗口
1. 作为扩张窗口的`ewm`窗口

在扩张窗口中，用户可以使用各类函数进行历史的累计指标统计，但这些内置的统计函数往往把窗口中的所有元素赋予了同样的权重。事实上，可以给出不同的权重来赋给窗口中的元素，指数加权窗口就是这样一种特殊的扩张窗口。

其中，最重要的参数是`alpha`，它决定了默认情况下的窗口权重为$w_i=(1−\alpha)^i,i\in\{0,1,...,t\}$，其中$i=t$表示当前元素，$i=0$表示序列的第一个元素。

从权重公式可以看出，离开当前值越远则权重越小，若记原序列为$x$，更新后的当前元素为$y_t$，此时通过加权公式归一化后可知：

$$
\begin{split}y_t &=\frac{\sum_{i=0}^{t} w_i x_{t-i}}{\sum_{i=0}^{t} w_i} \\
&=\frac{x_t + (1 - \alpha)x_{t-1} + (1 - \alpha)^2 x_{t-2} + ...
+ (1 - \alpha)^{t} x_{0}}{1 + (1 - \alpha) + (1 - \alpha)^2 + ...
+ (1 - \alpha)^{t}}\\\end{split}
$$

对于`Series`而言，可以用`ewm`对象如下计算指数平滑后的序列：

In [2]:
np.random.seed(0)
s = pd.Series(np.random.randint(-1,2,30).cumsum())
s.head()

0   -1
1   -1
2   -2
3   -2
4   -2
dtype: int32

In [24]:
s.ewm(alpha=0.2).mean()

ExponentialMovingWindow [com=4.0,min_periods=1,adjust=True,ignore_na=False,axis=0]

请用`expanding`窗口实现。

In [53]:
def ewmean(x, alpha):
    weight=list((1-alpha)**i for i in range(x.shape[0]-1,-1,-1)) 
    # 传入值x是Series，Series与Series是索引对应相乘，Series与列表按顺序相乘
    return ((x*weight)/sum(weight)).sum()
alpha=0.2
s.expanding().apply(lambda x:ewmean(x,alpha))

0    -1.000000
1    -1.000000
2    -1.409836
3    -1.609756
4    -1.725845
5    -1.529101
6    -1.648273
7    -1.492481
8    -1.609720
9    -1.921223
10   -2.376048
11   -2.510047
12   -2.613738
13   -2.485343
14   -2.177441
15   -2.140925
16   -2.112091
17   -2.089261
18   -2.071148
19   -2.056753
20   -2.247158
21   -2.398846
22   -2.720978
23   -3.178945
24   -3.544537
25   -3.635906
26   -3.909386
27   -3.927544
28   -4.142368
29   -4.314107
dtype: float64

2. 作为滑动窗口的`ewm`窗口

从第1问中可以看到，`ewm`作为一种扩张窗口的特例，只能从序列的第一个元素开始加权。现在希望给定一个限制窗口`n`，只对包含自身的最近的`n`个元素作为窗口进行滑动加权平滑。请根据滑窗函数，给出新的`wi`与`yt`的更新公式，并通过`rolling`窗口实现这一功能。

In [54]:
s.rolling(n).apply(lambda x:ewmean(x,alpha))

0          NaN
1          NaN
2    -1.409836
3    -1.737705
4    -2.000000
5    -1.590164
6    -1.672131
7    -1.327869
8    -1.672131
9    -2.147541
10   -3.147541
11   -3.327869
12   -3.262295
13   -2.590164
14   -1.852459
15   -1.672131
16   -1.737705
17   -2.000000
18   -2.000000
19   -2.000000
20   -2.409836
21   -2.737705
22   -3.409836
23   -4.147541
24   -4.737705
25   -4.590164
26   -4.672131
27   -4.327869
28   -4.672131
29   -4.737705
dtype: float64